In [1]:
using JuMP, Cbc
o = Model(with_optimizer(Cbc.Optimizer))

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[1]:2
└ @ Core In[1]:2


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

In [2]:
#viewers in millions in hours 9:21
v = [281.7
422
423
376.8
377.8
518.1
528
529
549.6
858.5
859.5
860.5
861.5]

13-element Array{Float64,1}:
 281.7
 422.0
 423.0
 376.8
 377.8
 518.1
 528.0
 529.0
 549.6
 858.5
 859.5
 860.5
 861.5

In [3]:
# qualifications variables
@variable(o, m_qual1[1:2, 9:21], Bin)      #men's qualification subdivision 1
@variable(o, m_qual2[1:2, 9:21], Bin)      #men's qualification subdivision 2
@variable(o, m_qual3[1:2, 9:21], Bin)      #men's qualification subdivision 3
@variable(o, w_qual1[1:2, 9:21], Bin)      #women's qualification subdivision 1
@variable(o, w_qual2[1:2, 9:21], Bin)      #women's qualification subdivision 2
@variable(o, w_qual3[1:2, 9:21], Bin)      #women's qualification subdivision 3

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Base.OneTo(2)
    Dimension 2, 9:21
And data, a 2×13 Array{VariableRef,2}:
 w_qual3[1,9]  w_qual3[1,10]  w_qual3[1,11]  …  w_qual3[1,20]  w_qual3[1,21]
 w_qual3[2,9]  w_qual3[2,10]  w_qual3[2,11]     w_qual3[2,20]  w_qual3[2,21]

In [4]:
# QUALIFIERS

#the events cannot take place at the same time
for day in 1:2
    for hour in 9:21
        @constraint(o, sum(m_qual1[day, hour] + m_qual2[day, hour] + m_qual3[day, hour]) <= 1)
        @constraint(o, sum(w_qual1[day, hour] + w_qual2[day, hour] + w_qual3[day, hour]) <= 1)
    end
end

#make sure every event takes 2hrs, each gender is on a different day 
@variable(o, g, Bin)
men_qual = [m_qual1, m_qual2, m_qual3]
women_qual= [w_qual1, w_qual2, w_qual3]
for event in men_qual
    @constraint(o, sum(event[1,:]) == 2-2*g)
    @constraint(o, sum(event[2,:]) == 2*g)
end

for event in women_qual
    @constraint(o, sum(event[1,:]) == 2*g)
    @constraint(o, sum(event[2,:]) == 2-2*g)
end

#events last 2 consecutive hours 
quals = [w_qual1, w_qual2, w_qual3, m_qual1, m_qual2, m_qual3]
for event in quals
    for h in 10:20
        @constraint(o, sum(event[1, h+1] + event[1, h-1]) >= event[1, h])
        @constraint(o, sum(event[2, h+1] + event[2, h-1]) >= event[2, h])
    end
    @constraint(o,event[1,9] <= event[1,10])
    @constraint(o,event[1,21] <= event[1,20])
    @constraint(o,event[2,9] <= event[2,10])
    @constraint(o,event[2,21] <= event[2,20])
end

#need 2 hrs inbetween qualifiying rounds
#men and women already on different days so this can be two seperate constraints
for d in 1:2
    for h in 9:18
        @constraint(o, sum(m_qual1[d, h] + m_qual2[d, h] + m_qual3[d, h] + m_qual1[d, h+1] + m_qual2[d, h+1] + m_qual3[d, h+1] + m_qual1[d, h+2] + m_qual2[d, h+2] + m_qual3[d, h+2] + m_qual1[d, h+3] + m_qual2[d, h+3] + m_qual3[d, h+3]) <=2)
        @constraint(o, sum(w_qual1[d, h] + w_qual2[d, h] + w_qual3[d, h] + w_qual1[d, h+1] + w_qual2[d, h+1] + w_qual3[d, h+1] + w_qual1[d, h+2] + w_qual2[d, h+2] + w_qual3[d, h+2] + w_qual1[d, h+3] + w_qual2[d, h+3] + w_qual3[d, h+3]) <=2)
    end
end

In [5]:
#test the code above

# objec = v'*(m_qual1[1,:])+v'*(m_qual1[2,:])+ v'*(m_qual2[1,:])+v'*(m_qual2[2,:])+ v'*(m_qual3[1,:])+v'*(m_qual3[2,:])
# #womens qual
# objec = objec + v'*(w_qual1[1,:])+v'*(w_qual1[2,:])+ v'*(w_qual2[1,:])+v'*(w_qual2[2,:])+ v'*(w_qual3[1,:])+v'*(w_qual3[2,:])

# @objective(o, Max, objec)

In [6]:
#optimize!(o)
# @show JuMP.value.(m_qual3[:,:])
# @show JuMP.value.(m_qual2[:,:])
# @show JuMP.value.(m_qual1[:,:])
# @show JuMP.value.(w_qual3[:,:])
# @show JuMP.value.(w_qual2[:,:])
# @show JuMP.value.(w_qual1[:,:])

In [7]:
# TEAMS FINALS
@variable(o, m_tf[4:5, 9:21], Bin)
@variable(o, w_tf[4:5, 9:21], Bin)

#team finals constraints
# men and women on different days
@variable(o, x, Bin)
@constraint(o, sum(m_tf[4, :]) == 2*x)
@constraint(o, sum(m_tf[5, :]) == 2-(2*x))
@constraint(o, sum(w_tf[4, :]) == 2-(2*x))
@constraint(o, sum(w_tf[5, :]) == 2*x)

#events last 2 consecutive hours 
finals = [m_tf, w_tf]
for event in finals
    for h in 10:20
        @constraint(o, sum(event[4, h+1] + event[4, h-1]) >= event[4, h])
        @constraint(o, sum(event[5, h+1] + event[5, h-1]) >= event[5, h])
    end
    @constraint(o,event[4,9] <= event[4,10])
    @constraint(o,event[4,21] <= event[4,20])
    @constraint(o,event[5,9] <= event[5,10])
    @constraint(o,event[5,21] <= event[5,20])
end


In [8]:
#only optimize teams finals(for testing purposes)
#@objective(o, Max ,v'*m_tf[4,:]+v'*m_tf[5,:]+v'*w_tf[4,:]+v'*w_tf[5,:])

In [9]:
#optimize!(o)

#@show JuMP.value.(w_tf[:,:])


In [10]:
# INDIVIDUAL ALL AROUND
@variable(o, m_ar[6:7, 9:21], Bin)
@variable(o, w_ar[6:7, 9:21], Bin)

# men and women on different days
@variable(o, d, Bin)
@constraint(o, sum(m_ar[6, :]) == 3*d)
@constraint(o, sum(m_ar[7, :]) == 3-(3*d))
@constraint(o, sum(w_ar[6, :]) == 2-(2*d))
@constraint(o, sum(w_ar[7, :]) == 2*d)

#2 consecutive for women, 3 for men
ar = [m_ar, w_ar]
for event in ar
    for h in 10:20
        @constraint(o, sum(event[6, h+1] + event[6, h-1]) >= event[6, h])
        @constraint(o, sum(event[7, h+1] + event[7, h-1]) >= event[7, h])
    end
    @constraint(o,event[6,9] <= event[6,10])
    @constraint(o,event[6,21] <= event[6,20])
    @constraint(o,event[7,9] <= event[7,10])
    @constraint(o,event[7,21] <= event[7,20])
end



In [11]:
objec = v'*m_ar[6,:]+v'*m_ar[7,:]+v'*w_ar[6,:]+v'*w_ar[7,:]
@objective(o, Max, objec)

281.7 m_ar[6,9] + 422 m_ar[6,10] + 423 m_ar[6,11] + 376.8 m_ar[6,12] + 377.8 m_ar[6,13] + 518.1 m_ar[6,14] + 528 m_ar[6,15] + 529 m_ar[6,16] + 549.6 m_ar[6,17] + 858.5 m_ar[6,18] + 859.5 m_ar[6,19] + 860.5 m_ar[6,20] + 861.5 m_ar[6,21] + 281.7 m_ar[7,9] + 422 m_ar[7,10] + 423 m_ar[7,11] + 376.8 m_ar[7,12] + 377.8 m_ar[7,13] + 518.1 m_ar[7,14] + 528 m_ar[7,15] + 529 m_ar[7,16] + 549.6 m_ar[7,17] + 858.5 m_ar[7,18] + 859.5 m_ar[7,19] + 860.5 m_ar[7,20] + 861.5 m_ar[7,21] + 281.7 w_ar[6,9] + 422 w_ar[6,10] + 423 w_ar[6,11] + 376.8 w_ar[6,12] + 377.8 w_ar[6,13] + 518.1 w_ar[6,14] + 528 w_ar[6,15] + 529 w_ar[6,16] + 549.6 w_ar[6,17] + 858.5 w_ar[6,18] + 859.5 w_ar[6,19] + 860.5 w_ar[6,20] + 861.5 w_ar[6,21] + 281.7 w_ar[7,9] + 422 w_ar[7,10] + 423 w_ar[7,11] + 376.8 w_ar[7,12] + 377.8 w_ar[7,13] + 518.1 w_ar[7,14] + 528 w_ar[7,15] + 529 w_ar[7,16] + 549.6 w_ar[7,17] + 858.5 w_ar[7,18] + 859.5 w_ar[7,19] + 860.5 w_ar[7,20] + 861.5 w_ar[7,21]

In [12]:
optimize!(o)
@show JuMP.value.(w_ar[:,:])
@show JuMP.value.(m_ar[:,:])

JuMP.value.(w_ar[:, :]) = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 6:7
    Dimension 2, 9:21
And data, a 2×13 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
JuMP.value.(m_ar[:, :]) = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 6:7
    Dimension 2, 9:21
And data, a 2×13 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0                 0.0                 0.0               
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.9999999999999999  0.9999999999999999  0.9999999999999999
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 4305 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 128 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 92 strengthened

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 6:7
    Dimension 2, 9:21
And data, a 2×13 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0

In [13]:
# specific events
@variable(o, m_floor[9:11, 9:21], Bin)      # mens floor
@variable(o, m_vault[9:11, 9:21], Bin)      # mens vault
@variable(o, m_pb[9:11, 9:21], Bin)         # mens parallel bars
@variable(o, m_pommel[9:11, 9:21], Bin)     # mens pommel horse
@variable(o, m_rings[9:11, 9:21], Bin)      # mens rings
@variable(o, m_horizontal[9:11, 9:21], Bin) #mens horizontal bar
@variable(o, w_floor[9:11, 9:21], Bin)      # womens floor
@variable(o, w_vault[9:11, 9:21], Bin)      # womens vault
@variable(o, w_ub[9:11, 9:21], Bin)         # womens uneven bars
@variable(o, w_b[9:11, 9:21], Bin)          # womens beam


@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_rings[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_rings[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i] for i in 9:21) <=4)

@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) >=3)
@constraint(o, sum(m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_rings[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i] for i in 9:21) <=4)


#at most two specific events can happen in an hour
#for each day of specific events, the sum of events in an hour cannot be more than 2
for i = 9:21
    @constraint(o, (m_floor[9,i]+m_vault[9,i]+m_pb[9,i]+m_pommel[9,i]+m_horizontal[9,i]+ w_floor[9,i]+w_vault[9,i]+w_ub[9,i]+ w_b[9,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[10,i]+m_vault[10,i]+m_pb[10,i]+m_pommel[10,i]+m_horizontal[10,i]+ w_floor[10,i]+w_vault[10,i]+w_ub[10,i]+ w_b[10,i]) <=2)
end

for i = 9:21
    @constraint(o, (m_floor[11,i]+m_vault[11,i]+m_pb[11,i]+m_pommel[11,i]+m_horizontal[11,i]+ w_floor[11,i]+w_vault[11,i]+w_ub[11,i]+ w_b[11,i]) <=2)
    end

#if womens floor is happening, mens floor cannot be happening (there is only one floor)
for i = 9:11
    for j = 9:21
        @constraint(o, (m_floor[i,j]+ w_floor[i,j] <=1))
    end
end

#women's and men's events cannot be at the same time
for day in 9:11
    for hour in 9:21
        @constraint(o, sum(w_floor[day, hour]+ w_vault[day, hour] + w_ub[day, hour] + w_b[day, hour]) <= 1)
        @constraint(o, sum(m_floor[day, hour]+ m_vault[day, hour] + m_pommel[day, hour] + m_horizontal[day, hour] + m_pb[day, hour] + m_rings[day, hour]) <= 1)
    end
end


In [14]:

#add up viewers at possible times of events to get objective to be maximized
#mens qual
objec = v'*(m_qual1[1,:])+v'*(m_qual1[2,:])+ v'*(m_qual2[1,:])+v'*(m_qual2[2,:])+ v'*(m_qual3[1,:])+v'*(m_qual3[2,:])
#womens qual
objec = objec + v'*(w_qual1[1,:])+v'*(w_qual1[2,:])+ v'*(w_qual2[1,:])+v'*(w_qual2[2,:])+ v'*(w_qual3[1,:])+v'*(w_qual3[2,:])
#team finals
objec = objec +v'*m_tf[4,:]+v'*m_tf[5,:]+v'*w_tf[4,:]+v'*w_tf[5,:]
#indiv all around
objec = objec +v'*m_ar[6,:]+v'*m_ar[7,:]+v'*w_ar[6,:]+v'*w_ar[7,:]

#specific events
#always a day of rest before finals (day 8)
objec = sum(v'*m_floor[i,:] for i in 9:11)
objec = objec +sum(v'*m_vault[i,:] for i in 9:11)
objec = objec +sum(v'*m_pb[i,:] for i in 9:11)
objec = objec +sum(v'*m_pommel[i,:] for i in 9:11)
objec = objec +sum(v'*m_horizontal[i,:] for i in 9:11)
objec = objec +sum(v'*m_rings[i,:] for i in 9:11)
objec = objec +sum(v'*w_floor[i,:] for i in 9:11)
objec = objec +sum(v'*w_vault[i,:] for i in 9:11)
objec = objec +sum(v'*w_ub[i,:] for i in 9:11)
objec = objec +sum(v'*w_b[i,:] for i in 9:11)


# #objective

# @objective(o, Max,  objec)
# #take a look at our model
# #o

# optimize!(o)

281.7 m_floor[9,9] + 422 m_floor[9,10] + 423 m_floor[9,11] + 376.8 m_floor[9,12] + 377.8 m_floor[9,13] + 518.1 m_floor[9,14] + 528 m_floor[9,15] + 529 m_floor[9,16] + 549.6 m_floor[9,17] + 858.5 m_floor[9,18] + 859.5 m_floor[9,19] + 860.5 m_floor[9,20] + 861.5 m_floor[9,21] + 281.7 m_floor[10,9] + 422 m_floor[10,10] + 423 m_floor[10,11] + 376.8 m_floor[10,12] + 377.8 m_floor[10,13] + 518.1 m_floor[10,14] + 528 m_floor[10,15] + 529 m_floor[10,16] + 549.6 m_floor[10,17] + 858.5 m_floor[10,18] + 859.5 m_floor[10,19] + 860.5 m_floor[10,20] + 861.5 m_floor[10,21] + 281.7 m_floor[11,9] + 422 m_floor[11,10] + 423 m_floor[11,11] + 376.8 m_floor[11,12] + 377.8 m_floor[11,13] + 518.1 m_floor[11,14] + 528 m_floor[11,15] + 529 m_floor[11,16] + 549.6 m_floor[11,17] + 858.5 m_floor[11,18] + 859.5 m_floor[11,19] + 860.5 m_floor[11,20] + 861.5 m_floor[11,21] + 281.7 m_vault[9,9] + 422 m_vault[9,10] + 423 m_vault[9,11] + 376.8 m_vault[9,12] + 377.8 m_vault[9,13] + 518.1 m_vault[9,14] + 528 m_vault[9,15

In [15]:
#look at time slots of qualifiers
# @show JuMP.value.(m_qual3[:,:])
# @show JuMP.value.(m_qual2[:,:])
# @show JuMP.value.(m_qual1[:,:])
# @show JuMP.value.(w_qual3[:,:])
# @show JuMP.value.(w_qual2[:,:])
# @show JuMP.value.(w_qual1[:,:])

In [16]:
#potential other objectives
#maxmin viewers for each section
m_qual_views = v'*(m_qual1[1,:])+v'*(m_qual1[2,:])+ v'*(m_qual2[1,:])+v'*(m_qual2[2,:])+ v'*(m_qual3[1,:])+v'*(m_qual3[2,:])
w_qual_views = v'*(w_qual1[1,:])+v'*(w_qual1[2,:])+ v'*(w_qual2[1,:])+v'*(w_qual2[2,:])+ v'*(w_qual3[1,:])+v'*(w_qual3[2,:])

team_final_views = v'*m_tf[4,:]+v'*m_tf[5,:]+v'*w_tf[4,:]+v'*w_tf[5,:]

indiv_views = v'*m_ar[6,:]+v'*m_ar[7,:]+v'*w_ar[6,:]+v'*w_ar[7,:]

#event views
e = sum(v'*m_floor[i,:] for i in 9:11)
e += sum(v'*m_vault[i,:] for i in 9:11)
e += sum(v'*m_pb[i,:] for i in 9:11)
e += sum(v'*m_pommel[i,:] for i in 9:11)
e += sum(v'*m_horizontal[i,:] for i in 9:11)
e += sum(v'*m_rings[i,:] for i in 9:11)
e +=sum(v'*w_floor[i,:] for i in 9:11)
e += sum(v'*w_vault[i,:] for i in 9:11)
e += sum(v'*w_ub[i,:] for i in 9:11)
e += sum(v'*w_b[i,:] for i in 9:11)

section_views = [m_qual_views,w_qual_views,team_final_views,indiv_views,e]
min_views = m_qual_views
for views in section_views
    @constraint(o, min_views <= views)
end
@objective(o, Max,  min_views)
optimize!(o)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3444 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 128 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 92 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 54 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 50 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 50 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 50 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 11 strengthened rows, 0 substitutions
Cgl0004I processed model has 531 rows, 458 columns (458 integer (458 of which binary)) and 3404 elements
Cbc0031I 7 added row